# **Tarefa 2 – Web Scraping em Ambiente Real (30%)**
### **Disciplina**: Coleta, Preparação e Análise de Dados - Prof. Lucas Pessutto

#### **Integrantes do grupo**: Ana Cristina Schmidt e Jeniffer Borges

In [12]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import re
from PIL import Image
import requests
import io
import json
import time


from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time


In [13]:
driver = webdriver.Chrome()

In [14]:
driver.get("https://www.imdb.com/chart/top/?ref_=nv_mv_250")

In [15]:
#driver.maximize_window()

filmes_info = []

In [16]:
titulos_filmes = []
titulos = driver.find_elements(By.XPATH, "//h3[@class='ipc-title__text' and number(substring-before(., '.')) ]")

for titulo in titulos:
    titulos_filmes.append(titulo.text)

In [17]:
duracoes_filmes = []
duracao_filmes = driver.find_elements(By.CLASS_NAME, "sc-b189961a-8")

tempo_pattern = re.compile(r'(\d{1}h( \d{1,2}m)?)|(\d{1,2}m)')

for duracao in duracao_filmes:
    if tempo_pattern.match(duracao.text):
        duracoes_filmes.append(duracao.text)

In [18]:
notas_filmes = []
notas = driver.find_elements(By.CLASS_NAME, "ipc-rating-star--rating")

nota_pattern = re.compile(r'\d+,\d')

for nota in notas:
    if nota_pattern.match(nota.text):
        notas_filmes.append(nota.text)

In [19]:
imagens_filmes = []
imagens = driver.find_elements(By.XPATH, "//img[@class='ipc-image']")

qnd_filme = 250

for i, imagem in enumerate(imagens[:qnd_filme]):
    imagem_url = imagem.get_attribute('src')
    
    imagens_filmes.append(imagem_url)
    
    #imagem_arq = requests.get(imagem_url).content
    #imagem = Image.open(io.BytesIO(imagem_arq))
    #display(imagem) ##### para capturar as imagens, ative essas 3 linhas, demora mais se ativar
    
    filme = {
            "titulo": titulos_filmes[i],
            "imagem_url": imagem_url,
            "duracao": duracoes_filmes[i] if i < len(duracoes_filmes) else "N/A",
            "nota": notas_filmes[i] if i < len(notas_filmes) else "N/A"
        }

    filmes_info.append(filme)
    


In [20]:
nome_arq = "filmes"

with open(f'{nome_arq}.json', 'w', encoding='utf-8') as json_file:
    json.dump(filmes_info, json_file, ensure_ascii=False, indent=4)
    
print(f"Informações dos filmes salvas no arquivo '{nome_arq}.json'.")
    

Informações dos filmes salvas no arquivo 'filmes.json'.


In [21]:
links = driver.find_elements(By.CLASS_NAME, "ipc-title-link-wrapper")

urls = [link.get_attribute('href') for link in links[:3]] ###limitado em 3, pra nao fazer 250 xd

todos_os_elencos = []

for url in urls:
    driver.get(url)
    time.sleep(2)

    try:
        elenco_equipe_link = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '/html/body/div[2]/main/div/section[1]/div/section/div/div[1]/section[4]/ul/li[3]/a[1]'))
        )

        driver.execute_script("arguments[0].scrollIntoView(true);", elenco_equipe_link)
        time.sleep(5)

        try:
            iframe = driver.find_element(By.TAG_NAME, "iframe")
            driver.execute_script("arguments[0].style.display = 'none';", iframe)
        except:
            print("No iframe found to close")

        action = ActionChains(driver)
        action.move_to_element(elenco_equipe_link).click().perform()

        time.sleep(2)

        atores = driver.find_elements(By.XPATH, '//table[@class="cast_list"]//tr//td[2]//a')
        personagens_td = driver.find_elements(By.XPATH, '//table[@class="cast_list"]//tr//td[@class="character"]')

        elencos = []
        for ator, td in zip(atores, personagens_td):
            personagens_links = td.find_elements(By.TAG_NAME, 'a')

            personagens = [a.text for a in personagens_links]

            elencos.append({
                'ator': ator.text,
                'personagens': personagens
            })

        todos_os_elencos.append({
            'url': url,
            'elenco': elencos
        })

    except Exception as e:
        print(f"Erro na URL {url}: {e}")

for elenco in todos_os_elencos:
    print(f"Elenco para o URL: {elenco['url']}")
    for ator in elenco['elenco']:
        print(f"Ator: {ator['ator']} - Personagens: {' / '.join(ator['personagens'])}")

Elenco para o URL: https://www.imdb.com/title/tt0111161/?ref_=chttp_t_1
Ator: Tim Robbins - Personagens: Andy Dufresne
Ator: Morgan Freeman - Personagens: Ellis Boyd 'Red' Redding
Ator: Bob Gunton - Personagens: Warden Norton
Ator: William Sadler - Personagens: Heywood
Ator: Clancy Brown - Personagens: Captain Hadley
Ator: Gil Bellows - Personagens: Tommy
Ator: Mark Rolston - Personagens: Bogs Diamond
Ator: James Whitmore - Personagens: Brooks Hatlen
Ator: Jeffrey DeMunn - Personagens: 1946 D.A.
Ator: Larry Brandenburg - Personagens: Skeet
Ator: Neil Giuntoli - Personagens: Jigger
Ator: Brian Libby - Personagens: Floyd
Ator: David Proval - Personagens: Snooze
Ator: Joseph Ragno - Personagens: Ernie
Ator: Jude Ciccolella - Personagens: Guard Mert
Ator: Paul McCrane - Personagens: Guard Trout
Ator: Renee Blaine - Personagens: Andy Dufresne's Wife
Ator: Scott Mann - Personagens: Glenn Quentin
Ator: John Horton - Personagens: 1946 Judge
Ator: Gordon Greene - Personagens: 1947 Parole Hearin

In [22]:
nome_arq = "elenco_filmes"

with open(f'{nome_arq}.json', 'w', encoding='utf-8') as json_file:
    json.dump(todos_os_elencos, json_file, ensure_ascii=False, indent=4)
    
print(f"Informações dos elencos salvas no arquivo '{nome_arq}.json'.")

Informações dos elencos salvas no arquivo 'elenco_filmes.json'.
